In [1]:
%run 02-connect.ipynb

In [2]:
%sql USE classicmodels

""


First, create a new table named employees_audit to keep the changes to the employees table:

In [3]:
%%sql

CREATE TABLE employees_audit (
    id INT AUTO_INCREMENT PRIMARY KEY,
    employeeNumber INT NOT NULL,
    lastname VARCHAR(50) NOT NULL,
    changedat DATETIME DEFAULT NULL,
    action VARCHAR(50) DEFAULT NULL
)

""


Next, create a BEFORE UPDATE trigger that is invoked before a change is made to the employees table.

In [4]:
%%sql

CREATE TRIGGER before_employee_update 
    BEFORE UPDATE ON employees
    FOR EACH ROW 
 INSERT INTO employees_audit
 SET action = 'update',
     employeeNumber = OLD.employeeNumber,
     lastname = OLD.lastname,
     changedat = NOW()

""


Inside the body of the trigger, we used the OLD keyword to access values of the columns employeeNumber and lastname of the row affected by the trigger.

In [5]:
%%sql

SHOW TRIGGERS

,Trigger,Event,Table,Statement,Timing,Created,sql_mode,Definer,character_set_client,collation_connection,Database Collation
0,before_employee_update,UPDATE,employees,"b""INSERT INTO employees_audit\n SET action = '...",BEFORE,2023-03-03 04:55:07.350,{NO_ENGINE_SUBSTITUTION},admin@%,utf8mb4,utf8mb4_general_ci,latin1_swedish_ci


After that, update a row in the employees table:

In [6]:
%%sql

UPDATE employees 
SET 
    lastName = 'Phan'
WHERE
    employeeNumber = 1056

""


Finally, query the employees_audit table to check if the trigger was fired by the UPDATE statement:

In [7]:
%%sql

SELECT * FROM employees_audit

,id,employeeNumber,lastname,changedat,action
0,1,1056,Patterson,2023-03-03 04:55:48,update


As you see clearly from the output, the trigger was automatically invoked and inserted a new row into the employees_audit table.

First, create a table called billings for demonstration:

In [8]:
%%sql

CREATE TABLE billings (
    billingNo INT AUTO_INCREMENT,
    customerNo INT,
    billingDate DATE,
    amount DEC(10 , 2 ),
    PRIMARY KEY (billingNo)
)

""


Second, create a new trigger called BEFORE UPDATE that is associated with the billings table:

In [9]:
%%sql

CREATE TRIGGER before_billing_update
    BEFORE UPDATE 
    ON billings FOR EACH ROW
BEGIN
    IF new.amount > old.amount * 10 THEN
        SIGNAL SQLSTATE '45000' 
            SET MESSAGE_TEXT = 'New amount cannot be 10 times greater than the current amount.';
    END IF;
END

""


The trigger activates before any update. If the new amount is 10 times greater than the current amount, the trigger raises an error.

Third, show the triggers:

In [10]:
%%sql

SHOW TRIGGERS

,Trigger,Event,Table,Statement,Timing,Created,sql_mode,Definer,character_set_client,collation_connection,Database Collation
0,before_billing_update,UPDATE,billings,"b""BEGIN\n IF new.amount > old.amount * 10 T...",BEFORE,2023-03-03 04:57:58.150,{NO_ENGINE_SUBSTITUTION},admin@%,utf8mb4,utf8mb4_general_ci,latin1_swedish_ci
1,before_employee_update,UPDATE,employees,"b""INSERT INTO employees_audit\n SET action = '...",BEFORE,2023-03-03 04:55:07.350,{NO_ENGINE_SUBSTITUTION},admin@%,utf8mb4,utf8mb4_general_ci,latin1_swedish_ci


Fourth, drop the before_billing_update trigger:

In [11]:
%%sql

DROP TRIGGER before_billing_update

""


Finally, show the triggers again to verify the removal:

In [12]:
%%sql

SHOW TRIGGERS

,Trigger,Event,Table,Statement,Timing,Created,sql_mode,Definer,character_set_client,collation_connection,Database Collation
0,before_employee_update,UPDATE,employees,"b""INSERT INTO employees_audit\n SET action = '...",BEFORE,2023-03-03 04:55:07.350,{NO_ENGINE_SUBSTITUTION},admin@%,utf8mb4,utf8mb4_general_ci,latin1_swedish_ci


## BEFORE INSERT Trigger

First, create a new table called WorkCenters:

In [13]:
%%sql

DROP TABLE IF EXISTS WorkCenters;

CREATE TABLE WorkCenters (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    capacity INT NOT NULL
);

""


Second, create another table called WorkCenterStats that stores the summary of the capacity of the work centers:

In [14]:
%%sql

DROP TABLE IF EXISTS WorkCenterStats;

CREATE TABLE WorkCenterStats(
    totalCapacity INT NOT NULL
);

""


The following trigger updates the total capacity in the WorkCenterStats table before a new work center is inserted into the WorkCenter table:

In [15]:
%%sql

CREATE TRIGGER before_workcenters_insert
BEFORE INSERT
ON WorkCenters FOR EACH ROW
BEGIN
    DECLARE rowcount INT;
    
    SELECT COUNT(*) 
    INTO rowcount
    FROM WorkCenterStats;
    
    IF rowcount > 0 THEN
        UPDATE WorkCenterStats
        SET totalCapacity = totalCapacity + new.capacity;
    ELSE
        INSERT INTO WorkCenterStats(totalCapacity)
        VALUES(new.capacity);
    END IF;
END

""


In this trigger:

- First, the name of the trigger is before_workcenters_insert specified in the CREATE TRIGGER clause.
- Second, the triggering event `before insert` is mentioned.
- Third, the table that the trigger associated with is WorkCenters table.
- Finally, inside the trigger body, we check if there is any row in the WorkCenterStats table.

If the table WorkCenterStats has a row, the trigger adds the capacity to the totalCapacity column. Otherwise, it inserts a new row into the WorkCenterStats table.

First, insert a new row into the WorkCenter table:

In [16]:
%%sql

INSERT INTO WorkCenters(name, capacity)
VALUES('Mold Machine',100)

""


Second, query data from the WorkCenterStats table:

In [17]:
%%sql

SELECT * FROM WorkCenterStats

,totalCapacity
0,100


The trigger has been invoked and inserted a new row into the WorkCenterStats table.

Third, insert a new work center:

In [18]:
%%sql

INSERT INTO WorkCenters(name, capacity)
VALUES('Packing',200)

""


Finally, query data from the WorkCenterStats:

In [19]:
%%sql

SELECT * FROM WorkCenterStats

,totalCapacity
0,300


The trigger has updated the total capacity from 100 to 200 as expected.

Note that to properly maintain the summary table WorkCenterStats, you should also create triggers to handle update and delete events on the WorkCenters table.

## AFTER INSERT Trigger

First, create a new table called members:

In [20]:
%%sql

DROP TABLE IF EXISTS members;

CREATE TABLE members (
    id INT AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(255),
    birthDate DATE,
    PRIMARY KEY (id)
);

""


Second, create another table called reminders that stores reminder messages to members.

In [21]:
%%sql

DROP TABLE IF EXISTS reminders;

CREATE TABLE reminders (
    id INT AUTO_INCREMENT,
    memberId INT,
    message VARCHAR(255) NOT NULL,
    PRIMARY KEY (id , memberId)
);

""


The following statement creates an AFTER INSERT trigger that inserts a reminder into the reminders table if the birth date of the member is NULL.

In [22]:
%%sql

CREATE TRIGGER after_members_insert
AFTER INSERT
ON members FOR EACH ROW
BEGIN
    IF NEW.birthDate IS NULL THEN
        INSERT INTO reminders(memberId, message)
        VALUES(new.id,CONCAT('Hi ', NEW.name, ', please update your date of birth.'));
    END IF;
END

""


In this trigger:

- First, the name of the trigger is after_members_insert specified in the CREATE TRIGGER clause.
- Second, the triggering event is set.
- Third, the table that the trigger associated with is members table.
- Finally, inside the trigger body, insert a new row into the reminder table if the birth date of the member is NULL.

**Testing the MySQL AFTER INSERT trigger**

First, insert two rows into the members table:

In [23]:
%%sql

INSERT INTO members(name, email, birthDate)
VALUES
    ('John Doe', 'john.doe@example.com', NULL),
    ('Jane Doe', 'jane.doe@example.com','2000-01-01');

""


Second, query data from the members table:

In [24]:
%%sql

SELECT * FROM members

,id,name,email,birthDate
0,1,John Doe,john.doe@example.com,None
1,2,Jane Doe,jane.doe@example.com,2000-01-01


Third, query data from reminders table:

In [25]:
%%sql

SELECT * FROM reminders

,id,memberId,message
0,1,1,"Hi John Doe, please update your date of birth."


We inserted two rows into the members table. However, only the first row that has a birth date value NULL, therefore, the trigger inserted only one row into the reminders table.